Import SysMLv2 Model:

In [5]:
import syside
import os

MAIN_DIR = "/Users/alejandronietocuatenta/Documents/sysmlv2_omg_introduction/"
# MODEL_DIR = "omg_single_example/"
# MODEL_DIR = "omg_introduction/" # mid complexity example
MODEL_DIR = "omg_simple_vehicle" # high complexity example

# Construct full directory path
full_dir = os.path.join(MAIN_DIR, MODEL_DIR)

# Find the first file ending with .sysml
for file in os.listdir(full_dir):
    if file.endswith(".sysml"):
        MODEL_FILE_PATH = os.path.join(full_dir, file)
        break  # stop after finding the first .sysml file

(model, diagnostics) = syside.load_model([MODEL_FILE_PATH])

Check Metamodel structure:

In [6]:
for doc in model.user_docs:
    print(doc)
    with doc.lock() as locked:
        print(f"Model sexp:\n {syside.sexp(locked.root_node)}")

Model sexp:
 (Namespace
  (OwningMembership
    (Package SimpleVehicleModel
      (NamespaceImport)
      (NamespaceImport)
      (OwningMembership
        (Package Definitions
          (NamespaceImport)
          (NamespaceImport)
          (NamespaceImport)
          (NamespaceImport)
          (NamespaceImport)
          (NamespaceImport)
          (NamespaceImport)
          (NamespaceImport)
          (NamespaceImport)
          (NamespaceImport)
          (NamespaceImport)
          (MembershipImport)
          (NamespaceImport)
          (OwningMembership
            (Package PartDefinitions
              (OwningMembership
                (PartDefinition Vehicle
                  (Subclassification)
                  (FeatureMembership
                    (AttributeUsage mass
                      (Subsetting)))
                  (FeatureMembership
                    (AttributeUsage dryMass
                      (Subsetting)))
                  (FeatureMembership
             

Check PartUsage:

In [7]:
for PartUsage in model.nodes(syside.PartUsage):
    print(PartUsage)
    print(f"\tPartUsage::part_definitions: {len(list(PartUsage.part_definitions))}")
    print(f"\tItemUsage::item_definitions: {len(list(PartUsage.part_definitions))}")
    print(f"\tOccurrenceUsage::individual_definition: {PartUsage.individual_definition}")
    print(f"\tElement::declared_name: {PartUsage.declared_name}")
    print(f"\tElement::documentation: {len(list(PartUsage.documentation))}")
    print(f"\tElement::element_id: {PartUsage.element_id}")
    print(f"\tElement::owner: {PartUsage.owner}")
    print(f"\tElement::owning_membership: {PartUsage.owning_membership}")
    print(f"\tElement::owning_namespace: {PartUsage.owning_namespace}")
None

SimpleVehicleModel::VehicleLogicalConfiguration::PartsTree::vehicleLogical
	PartUsage::part_definitions: 2
	ItemUsage::item_definitions: 2
	OccurrenceUsage::individual_definition: None
	Element::declared_name: vehicleLogical
	Element::documentation: 0
	Element::element_id: 29449597-3c07-59fc-aa2b-968a6b054213
	Element::owner: SimpleVehicleModel::VehicleLogicalConfiguration::PartsTree
	Element::owning_membership: SimpleVehicleModel::VehicleLogicalConfiguration::PartsTree::vehicleLogical/owningMembership
	Element::owning_namespace: SimpleVehicleModel::VehicleLogicalConfiguration::PartsTree
SimpleVehicleModel::VehicleLogicalConfiguration::PartsTree::vehicleLogical::torqueGenerator
	PartUsage::part_definitions: 2
	ItemUsage::item_definitions: 2
	OccurrenceUsage::individual_definition: None
	Element::declared_name: torqueGenerator
	Element::documentation: 0
	Element::element_id: d7217846-bcf9-5b04-a794-707aaadf9ef1
	Element::owner: SimpleVehicleModel::VehicleLogicalConfiguration::PartsTree:

Export to JSON:

In [8]:
from pathlib import Path

assert len(model.user_docs) == 1

with model.user_docs[0].lock() as locked:
    json_output = syside.json.dumps(
        locked.root_node, syside.SerializationOptions.minimal()
    )

# Use current working directory (instead of __file__)
output_path = Path.cwd() / "export.json"

# Write JSON file
output_path.write_text(json_output, encoding="utf-8")

print(f"JSON saved to: {output_path}")


JSON saved to: /Users/alejandronietocuatenta/Documents/sysmlv2_omg_introduction/export.json


Create cypher script for neo4j:

In [11]:
import json

with open("export.json") as f:
    data = json.load(f)

cypher = []

# 1. Create all nodes
for e in data:
    props = []
    for key in ("@id", "@type", "declaredName", "memberName", "operator"):
        val = e.get(key)
        if val:
            field = "id" if key == "@id" else "type" if key == "@type" else key
            props.append(f'{field}:"{val}"')
    # cypher.append(f'CREATE (:`Node` {{{", ".join(props)}}});') # original
    # cypher.append(f'CREATE ({e.get("@type")}:Node {{{", ".join(props)}}});') # using @type as node label name
    cypher.append(f'CREATE (:{e.get("@type")} {{{", ".join(props)}}});') # using @type as node label

# 2. Create relationships
rels = {
    "owningRelatedElement": "OWNS",
    "ownedRelationship": "OWNS_REL",
    "memberElement": "MEMBER_OF",
    "ownedRelatedElement": "OWNS_RELATED",
    "owningRelationship": "OWNING_REL",
    "general": "GENERAL",
    "specific": "SPECIFIC",
    "type": "TYPE_OF",
    "typedFeature": "TYPED_FEATURE",
}
for e in data:
    src_id = e["@id"]
    for k, rel in rels.items():
        if k in e:
            targets = e[k] if isinstance(e[k], list) else [e[k]]
            for t in targets:
                tid = t.get("@id")
                if tid:
                    td_index = next((i for i, item in enumerate(data) if item["@id"] == t.get("@id")), None) # target data index
                    if td_index is None:
                        print(f"td_index is None! {e["@id"]} {e.get("@type")}")
                    else:    
                        cypher.append(
                            # f'MATCH (a:Node {{id:"{src_id}"}}), (b:Node {{id:"{tid}"}}) ' # original
                            f'MATCH (a:{e.get("@type")} {{id:"{src_id}"}}), (b:{data[td_index]['@type']} {{id:"{tid}"}}) ' #
                            f'CREATE (a)-[:{rel}]->(b);'
                        )

with open("export.cypher", "w") as f:
    f.write("\n".join(cypher))


td_index is None! 7f4603c3-0d17-49e7-947d-c2380a462707 Subsetting
td_index is None! d9d975f8-63f4-47af-9032-9bb66d202343 Subsetting
td_index is None! 2361acbc-3544-4596-ba5f-33976171d617 Subsetting
td_index is None! e683eb08-9407-4105-b9cd-d33305c5a0b9 Subsetting
td_index is None! 026ca7b8-c737-4f5e-80cb-210fd8ad984d Subsetting
td_index is None! 7dad40e6-cc20-4aa6-8cba-ad42c40a6b57 Subsetting
td_index is None! 080120b7-8ead-41e9-add4-ac7fb4a41140 Subsetting
td_index is None! dfc4e21b-6d3e-4bb7-8e6d-e4ebfabbdd06 Subsetting
td_index is None! 9cfc94ab-de0a-468c-a0b0-48d663f5d783 FeatureTyping
td_index is None! 9cfc94ab-de0a-468c-a0b0-48d663f5d783 FeatureTyping
td_index is None! c10a5d60-cfbc-48b3-9bff-06ce9c9c4cf4 FeatureTyping
td_index is None! c10a5d60-cfbc-48b3-9bff-06ce9c9c4cf4 FeatureTyping
td_index is None! 813be596-9287-4521-9428-e9139cee8aa9 Membership
td_index is None! 5169c052-6c48-45c9-97f4-956f00fb8cc0 Subsetting
td_index is None! 01ea42e7-3025-494f-9e3b-576c789dd6e1 Subsettin